# Solução Para o Trabalho Prático 01  <h1>

## Problema 02:<h3>
    
1. Implementação de um KEM-RSA que inicializa as instâncias com os respetivos paramêtros de segurança, gera chaves publicas e privadas, encapsulamento e revelação da chave gerada;
    
2. Segundo, ainda baseado no KEM implementado, a usufruindo da transformação de Fujisaki-Okamoto, a criação de um PKE sendo IND-CCA;
    
3. Em seguida, a implementação de um DSA, onde são definidos na inicialização os parametros dos primos _p_ e _q_ com seus respectivos tamanhos, e ainda assinatura digital e a verificação da mesma;
    
4. Por fim, a implementação de um ECDSA que utilizará uma das curvas primas definidas pelo FIPS186-4.    

In [1]:
""" Imports Necessários para as implementações """

import os
import random
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC
from sage.arith.power import generic_power

**Implementação KEM-RSA**

Temos então definidos a classe KEM-RSA, composta por 9 funções:

- _init_
- _generate_keys_
- _encapsulate_
- _reveal_
- _verify_
- _rsa_encrypt_
- _rsa_decrypt_
- _kdf_
- _print_nums_

descrição da classe e funções

In [2]:
class KEMRSA:
    # Construtor da Classe
    def __init__(self, seg):
        self.size = seg
        self.p_q_size = int(seg/2)
        self.kdf_salt = os.urandom(16)
        self.p = None 
        self.q = None 
        self.d = None
        self.n = None 
        self.e = None
        self.m = None
        self.m_b = None
        self.m_as_int = None
        self.m_b_as_int = None
        self.ct = None
        self.symetric_key_a = None
        self.symetric_key_b = None
        
    def generate_keys(self):
        # conjunto dos números primos
        P = Primes()
        
        # gerar um p
        p_candidate = ZZ.random_element(2^(self.p_q_size - 1) + 1, (2^self.p_q_size) - 1)
        while not (p_candidate in P):
            p_candidate = ZZ.random_element(2^(self.p_q_size - 1) + 1, (2^self.p_q_size) - 1)
            
        # gerar um q
        q_candidate = ZZ.random_element(2^(self.p_q_size - 1) + 1, (2^self.p_q_size) - 1)
        while not (q_candidate in P):
            q_candidate = ZZ.random_element(2^(self.p_q_size - 1) + 1, (2^self.p_q_size) - 1)
     
        # p e q escolhidos
        self.p = p_candidate
        self.q = q_candidate
        
        # calcular n
        self.n = self.p * self.q  
        
        # calcular phi
        phi = (self.p - 1)*(self.q - 1) 
        
        # gerar um e
        #e_candidate = ZZ.random_element(phi)
        #while gcd(e_candidate, phi) != 1:
        #    e_candidate = ZZ.random_element(phi)
        # e escolhido
        self.e = 65537 
        
        # algoritmo extendido de Euclides para calcular d
        bezout = xgcd(self.e, phi) 
        d_candidate = Integer(mod(bezout[1], phi))         
        self.d = d_candidate
        
    def encapsulate(self):
        self.m_as_int = int(ZZ.random_element(2^(self.size - 2)))
        self.m = self.m_as_int.to_bytes(int(self.size/8), "big")
        self.symetric_key_a = self.kdf(self.m)
        self.rsa_encrypt()
        
    def reveal(self):
        self.rsa_decrypt()
        self.m_b = int(self.m_b_as_int).to_bytes(int(self.size/8), "big")
        self.symetric_key_b = self.kdf(self.m_b)
        
    def verify(self):
        #print("Key A:", self.symetric_key_a)
        #print("Key B:", self.symetric_key_b)
        #print("M Int A:", self.m_as_int)
        #print("M Int B:", self.m_b_as_int)
        #print(self.kdf_salt)
        print("Chaves iguais:", self.symetric_key_a == self.symetric_key_b)
    
    def rsa_encrypt(self):
        self.ct = power_mod(self.m_as_int, self.e, self.n)
    
    def rsa_decrypt(self):
        self.m_b_as_int = power_mod(self.ct, self.d, self.n)        
        
    def kdf(self, password):
        kdf = PBKDF2HMAC(
            algorithm=hashes.SHA256(),
            length=32,
            salt=self.kdf_salt,
            iterations=100000,
        )
        return kdf.derive(password)
    
    def print_nums(self):
        print("p =", self.p,"\n")
        print("q =", self.q,"\n")
        print("d =", self.d,"\n")
        print("n =", self.n,"\n")
        print("e =", self.e,"\n")

_Função para realizar a chamada dos metodos atribuidos na classe KEM-RSA_

In [3]:
def ex1():
    a = KEMRSA(128)
    a.generate_keys()
    a.encapsulate()
    a.reveal()
    a.verify()
    #a.print_nums()

**Implementação DSA**

A class DSA é composta pelas funções init, generate_parameters, generate_keys, sign, compute_s, compute_r, hash_init e verify, onde:

**descrição dos metodos**

In [4]:
class DSA:
    def __init__(self, seg):
        self.l = seg
        self.n = 256
        self.h = 2
        self.q = None
        self.p = None
        self.g = None
        self.private_key = None
        self.public_key = None
        self.r = None
        self.s = None
        self.k = None
        self.h_m  = None
        self.w = None
        self.u1 = None
        self.u2 = None
        self.v = None
        
    def generate_parameters(self):
        # conjunto dos números primos
        P = Primes()
        
        flag = 1
        
        while flag:
            # gerar o p
            p_candidate = ZZ.random_element(2^(self.l - 1) + 1, (2^self.l) - 1)
            while not (p_candidate in P):
                p_candidate = ZZ.random_element(2^(self.l - 1) + 1, (2^self.l) - 1)
            self.p = p_candidate
            
            # gerar o q
            p_menos_um = self.p - 1
            q_candidate = p_menos_um
            div = 2
            while not ((q_candidate in P)) & (q_candidate > 1):
                q_candidate = int(p_menos_um / div)
                div += 1
            if (q_candidate in P) & (q_candidate > 1):
                self.q = q_candidate
                flag = 0
                
        # gerar o g 
        self.g = power_mod(self.h, int((self.p - 1)/self.q), self.p)
        while self.g == 1:
            self.h = ZZ.random_element(3, p - 2)
            self.g = power_mod(self.h, int((self.p - 1)/self.q), self.p)
            
    def generate_key(self):
        self.private_key = ZZ.random_element(1, self.q-1)
        self.public_key = power_mod(self.g, self.private_key, self.p)
    
    def sign(self, message):
        self.k = ZZ.random_element(1, self.q-1)
        self.compute_r()
        self.compute_s(message)
        while (self.r == 0) | (self.s == 0):
            self.k = ZZ.random_element(1, self.q-1)
            self.compute_r()
            self.compute_s(message)
    
    def compute_r(self):
        tmp = power_mod(self.g, self.k, self.p)
        self.r = tmp % self.q
    
    def compute_s(self, message):
        tmp = self.hash_int(message)
        self.h_m = tmp
        tmp = tmp + self.private_key * self.r
        tmp = int(tmp/self.k)
        self.s = tmp % self.q
        
    def hash_int(self, message):
        digest = hashes.Hash(hashes.SHA256())
        digest.update(message)
        return int.from_bytes(digest.finalize(), "big")
            
    def verify(self, message):
        self.w = power_mod(self.s, -1, self.q)
        self.u1 = (self.h_m * self.w) % self.q
        self.u2 = (self.r * self.w) % self.q
        tmp = (generic_power(self.g, self.u1) * generic_power(self.y, self.u2)) % self.p
        self.v = tmp % self.q
        print(self.v == self.r)

_Função para realizar a chamada dos metodos atribuidos na classe KEM-RSA_

In [5]:
def ex3():
    a = DSA(512)
    a.generate_parameters()
    a.generate_key()
    m = os.urandom(16)
    a.sign(m)
    #a.verify(m)

**descrição**

In [ ]:
class DSA:
    def __init__(self):
        self.p = None
        self.n = None
        self.seed = None
        self.c = None
        self.b = None
        self.gx = None
        self.gy = None
        
    def curva_p192(self):
        self.p = 6277101735386680763835789423207666416083908700390324961279
        self.n = 6277101735386680763835789423176059013767194773182842284081
        self.seed = int(0x3045ae6fc8422f64ed579528d38120eae12196d5)
        self.c = int(0x3099d2bbbfcb2538542dcd5fb078b6ef5f3d6fe2c745de65)
        self.b = int(0x64210519e59c80e70fa7e9ab72243049feb8deecc146b9b1)
        self.gx = int(0x188da80eb03090f67cbf20eb43a18800f4ff0afd82ff1012)
        self.gy = int(0x07192b95ffc8da78631011ed6b24cdd573f977a11e794811)
        
    def curva_p224(self):
        self.p = 26959946667150639794667015087019630673557916260026308143510066298881
        self.n = 26959946667150639794667015087019625940457807714424391721682722368061
        self.seed = int(0xbd71344799d5c7fcdc45b59fa3b9ab8f6a948bc5)
        self.c = int(0x5b056c7e11dd68f40469ee7f3c7a7d74f7d121116506d031218291fb)
        self.b = int(0xb4050a850c04b3abf54132565044b0b7d7bfd8ba270b39432355ffb4)
        self.gx = int(0xb70e0cbd6bb4bf7f321390b94a03c1d356c21122343280d6115c1d21)
        self.gy = int(0xbd376388b5f723fb4c22dfe6cd4375a05a07476444d5819985007e34)
        
    def curva_p256(self):
        return 1
    
    def curva_p384(self):
        return 1
    
    def curva_p521(self):
        return 1
    
    def generate_parameters(self):
        
    def generate_key(self):
    
    def sign(self, message):
            
    def verify(self, message):

_Função para realizar a chamada dos metodos atribuidos na classe KEM-RSA_

In [6]:
ex2()